### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 1;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

### Utility functions

In [7]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [8]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [9]:
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        mse = logs.get("mse");
        if(mse < 0.005):
            print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
            self.model.stop_training = True;

In [10]:
def NN_model(dataset, test = None):
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES+1])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,12,43))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    actual = list(series_transformed[WINDOW_SIZE:, 0]);

    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    results, actual, pure_forecast = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_test, MSE_pure,\
        np.array(pure_forecast[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_test, M_pure, forecast = NN_test(ZONE, plot=False);
        print(ZONE, M_train, M_test, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0049 in round 76 and stopped training

71 71 12
Wall time: 6.45 s
1013 5.491106038140538 57.203350722052804 58.73997306968897 [1116.3482701  1112.49613883 1108.07303426 1103.0960691  1098.12485506
 1093.27753064 1088.58073169 1083.2560583  1078.26422771 1073.63011758
 1069.32599886 1065.31860567]

Got an mse at 0.0049 in round 61 and stopped training

71 71 12
Wall time: 5.75 s
1020 5.663806102229374 57.401098214442364 63.806940626092796 [1130.49474673 1128.34183074 1125.88799969 1123.47917011 1121.17163736
 1118.63097197 1115.45264068 1112.27979283 1109.15845942 1106.06944952
 1102.98573579 1099.88347105]

Got an mse at 0.0050 in round 165 and stopped training

71 71 12
Wall time: 7.69 s
1040 4.518641377981203 66.6551799982575 70.09484432456594 [1101.97177702 1074.705251   1071.42203728 1069.44786686 1067.70435307
 1066.10523833 1064.52985509 1062.89075073 1059.99190775 1057.425324
 1055.56776541 1054.52289605]

Got an mse at 0.0050 in round 196 and stopped training




Got an mse at 0.0050 in round 251 and stopped training

71 71 12
Wall time: 9.78 s
1845 6.72496020636546 50.09052771141643 47.1087769720099 [1815.31103548 1814.55816471 1814.06542854 1813.64876463 1813.27300199
 1812.88516087 1812.42292149 1811.81257281 1810.95504215 1809.71862906
 1807.9523428  1805.50842544]

Got an mse at 0.0050 in round 183 and stopped training

71 71 12
Wall time: 9.15 s
1850 9.676632815427421 110.16311685345043 108.10859327337961 [1993.93204712 1989.09427289 1984.37093037 1978.89342007 1973.30056924
 1967.87357678 1963.10484772 1959.5299564  1956.12554881 1950.8979508
 1947.79347923 1944.25638176]

Got an mse at 0.0050 in round 279 and stopped training

71 71 12
Wall time: 11.1 s
1851 8.574338943474741 124.08028508145094 127.79120670990525 [1994.59125466 1995.55294132 1994.35025639 1988.40132561 1973.79081885
 1950.51402216 1930.37667356 1918.51887799 1910.76048321 1903.19785313
 1893.75172501 1887.76203889]

Got an mse at 0.0050 in round 154 and stopped trainin

71 71 12
Wall time: 12.7 s
2124 9.702936879576903 145.43411229269714 144.52770959347896 [2297.77487238 2304.29573543 2304.86318309 2305.23766883 2305.69184578
 2306.33541446 2307.18584079 2308.17397416 2309.16247207 2309.96237803
 2310.34320344 2309.98361694]

Got an mse at 0.0050 in round 387 and stopped training

71 71 12
Wall time: 14.2 s
2125 9.613665775465751 194.69386993511367 204.91380192327563 [2481.78193861 2477.79565738 2474.86198898 2472.03837818 2469.40671544
 2466.86662919 2464.33366419 2461.75445258 2459.12192427 2456.4951226
 2454.0061762  2451.83185431]

Got an mse at 0.0050 in round 155 and stopped training

71 71 12
Wall time: 9.5 s
2126 10.416778810179263 203.79820045180102 207.44370410297077 [2276.16252155 2277.44590134 2279.89409345 2281.63277505 2281.57569529
 2279.53291416 2275.63063282 2271.6280243  2275.9306435  2279.35169791
 2282.26552135 2282.43755267]

Got an mse at 0.0050 in round 289 and stopped training

71 71 12
Wall time: 12.1 s
2127 8.76938449471193 1


Got an mse at 0.0049 in round 234 and stopped training

71 71 12
Wall time: 10.9 s
2171 7.7580844916628 56.20661106368634 67.42531585385694 [2182.33360193 2170.70105264 2159.87200008 2151.41715803 2147.84068903
 2150.37383599 2159.51063903 2172.73480641 2180.32332303 2182.9175361
 2181.61681287 2177.79471557]

Got an mse at 0.0049 in round 243 and stopped training

71 71 12
Wall time: 11.5 s
2176 8.965633282370042 82.23228529215642 86.32809964663363 [2281.8332631  2282.51788176 2280.81738771 2278.36689104 2275.30224573
 2271.7485535  2267.96391499 2264.33130295 2261.20756246 2258.75682542
 2256.91279142 2255.47112967]

Got an mse at 0.0049 in round 88 and stopped training

71 71 12
Wall time: 8.19 s
2184 14.3392715060433 64.41974065404536 67.89678530709008 [2156.46737004 2146.81777864 2140.55595816 2138.7251341  2142.57238049
 2146.66559544 2150.40882552 2153.67898906 2156.51168325 2159.01088921
 2161.25051748 2163.1451587 ]
71 71 12
Wall time: 17 s
2210 6.8603700679600825 129.0147229


Got an mse at 0.0050 in round 99 and stopped training

71 71 12
Wall time: 8.51 s
2861 5.528834252975675 37.17669965870439 37.73785603626618 [1208.70146176 1205.24079374 1201.34887378 1197.03665092 1192.0583162
 1186.41230705 1180.0200087  1173.1608562  1164.72520913 1155.10052891
 1144.96639164 1135.022007  ]

Got an mse at 0.0050 in round 176 and stopped training

71 71 12
Wall time: 9.89 s
2863 5.073020834263077 36.060324795481115 36.89439961372045 [1231.440048   1230.94079233 1230.07127338 1228.94932662 1227.57635482
 1225.95356638 1224.08464819 1221.56019851 1218.48812662 1215.27053682
 1211.97474764 1208.64732787]

Got an mse at 0.0049 in round 128 and stopped training

71 71 12
Wall time: 8.81 s
2864 5.500108471719838 40.23819169199673 50.02959147309547 [1293.82634912 1293.43426962 1289.36653237 1281.16016544 1270.48069229
 1257.72889372 1243.25027694 1227.67734494 1211.96170802 1197.18270075
 1184.2291537  1173.55472677]

Got an mse at 0.0049 in round 217 and stopped training


...


Got an mse at 0.0048 in round 33 and stopped training

71 71 12
Wall time: 7.2 s
92618 8.615549625930816 70.75233493039443 69.24410312046209 [2109.11280711 2106.68159515 2103.36973459 2099.68522075 2095.68603108
 2091.46279147 2087.1214797  2082.7741514  2078.5264183  2073.60512073
 2069.31576693 2065.52901326]

Got an mse at 0.0050 in round 478 and stopped training

71 71 12
Wall time: 16.2 s
92620 4.221108267180825 62.472598085177246 63.194165147494786 [1935.96000791 1931.7936752  1925.25813973 1916.67768026 1906.15010959
 1894.83316377 1884.89016875 1880.5571406  1878.16836592 1876.67635576
 1875.70540009 1875.03518839]

Got an mse at 0.0050 in round 51 and stopped training

71 71 12
Wall time: 7.25 s
92626 7.827870183902884 22.7306687830916 22.828438560149856 [2207.59254003 2208.22746421 2209.23253987 2209.61010427 2209.3735342
 2208.62279837 2208.4750147  2208.26444999 2207.89443244 2207.39691083
 2206.77988772 2206.04273648]

Got an mse at 0.0050 in round 54 and stopped training


Got an mse at 0.0050 in round 263 and stopped training

71 71 12
Wall time: 11.7 s
93308 2.267139731149964 17.20251567426063 17.351191213723723 [852.34665336 852.29542746 851.85284657 851.10025564 850.07021214
 848.74720863 846.86168045 843.57334208 839.99851504 836.4609482
 833.23416585 830.45980134]

Got an mse at 0.0050 in round 224 and stopped training

71 71 12
Wall time: 10.8 s
93309 4.009521357119111 43.73702669085005 42.04759498200158 [1032.55340029 1031.98730196 1030.72974493 1028.18006477 1025.1604359
 1023.3211708  1020.95369871 1017.57976879 1013.40550156 1008.40710895
 1003.69475237  999.5444434 ]

Got an mse at 0.0050 in round 35 and stopped training

71 71 12
Wall time: 7.04 s
93906 16.033601533781216 193.30729304905313 193.4996814341218 [1816.14160436 1811.89781061 1803.54305937 1789.35439894 1770.95349237
 1749.01859454 1723.60264682 1691.80987115 1652.6347814  1612.06916906
 1572.43597403 1531.93980214]

Got an mse at 0.0050 in round 72 and stopped training

71 71 12

71 71 12
Wall time: 9.1 s
94549 15.926370926016675 61.657123790558835 63.119133438652014 [2735.36874244 2735.69375093 2733.80951853 2729.1938987  2723.94555589
 2714.77951186 2703.32539105 2690.72492075 2677.28592167 2663.25683143
 2648.85695096 2634.29046793]

Got an mse at 0.0050 in round 40 and stopped training

71 71 12
Wall time: 7.25 s
94565 14.524330965364737 39.98624023893599 39.4166635985908 [1997.55074986 2002.45856881 2005.94646007 2008.52202154 2010.4169417
 2011.80789381 2012.78303525 2013.36316047 2013.54040672 2013.30765274
 2012.67278057 2011.66223672]

Got an mse at 0.0050 in round 451 and stopped training

71 71 12
Wall time: 15.4 s
94568 14.138948881502895 96.79062668965886 100.1339638541844 [2755.2310204  2749.06482848 2743.5227034  2738.32206918 2733.48804682
 2729.08929382 2725.18564525 2721.79996031 2718.91199865 2716.46899292
 2714.40173696 2712.63812655]

Got an mse at 0.0048 in round 37 and stopped training

71 71 12
Wall time: 6.96 s
94603 19.778725744803598 


Got an mse at 0.0050 in round 74 and stopped training

71 71 12
Wall time: 7.83 s
95928 11.705790846676937 323.985535362241 310.8626592303576 [1404.73244299 1401.22179293 1396.24205037 1387.53123007 1378.31553779
 1369.3471782  1358.49566027 1347.58379862 1336.52574365 1325.38570046
 1314.23602881 1303.18522158]

Got an mse at 0.0050 in round 181 and stopped training

71 71 12
Wall time: 10.1 s
96150 14.715739477833683 144.39600820692309 146.85655001248242 [1658.16456205 1646.06921392 1628.69368645 1609.39538597 1589.71323647
 1571.18736293 1558.52645264 1551.11437082 1547.33899654 1540.74369727
 1534.92935194 1529.79936043]

Got an mse at 0.0050 in round 263 and stopped training

71 71 12
Wall time: 11.9 s
96826 6.890442699327043 120.12692180036476 122.54456550809617 [1537.65531789 1535.11004056 1534.73781055 1534.04427435 1532.86186568
 1531.08652656 1528.17041125 1522.9434942  1517.39648248 1513.02987851
 1508.69271275 1504.55436929]

Got an mse at 0.0049 in round 60 and stopped tr


Got an mse at 0.0050 in round 112 and stopped training

71 71 12
Wall time: 8.39 s
98105 9.581168416208248 64.0019064414766 63.76051385505382 [1846.50653069 1846.6291216  1844.72880689 1841.16711056 1835.66197005
 1828.02750203 1818.29186656 1807.30669335 1796.11850503 1785.76792495
 1776.77701913 1768.63153646]

Got an mse at 0.0050 in round 130 and stopped training

71 71 12
Wall time: 8.95 s
98107 7.337105234319059 65.47319896691852 65.22803335157899 [1967.32176395 1964.1394726  1960.68481044 1957.24582207 1953.7641887
 1949.10911158 1938.0488443  1924.24598734 1911.73289095 1901.59429882
 1893.47618767 1886.81884136]

Got an mse at 0.0050 in round 79 and stopped training

71 71 12
Wall time: 8.09 s
98109 12.334987055015214 143.21725705935287 141.03580302444755 [2102.76742198 2094.93701495 2084.35679948 2068.68341086 2050.46299937
 2029.80108365 2006.90614232 1982.12365542 1955.93512522 1932.654553
 1910.29081716 1890.55298533]

Got an mse at 0.0049 in round 126 and stopped trainin

In [14]:
print(len(score_dict["RMSE_pure"]), sum(score_dict["RMSE_test"])/len(score_dict["RMSE_test"]),
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

1301 61.29578374688722 62.93960344217968


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_test,RMSE_pure,forecast
0,1013,5.491106,57.203351,58.739973,"[1116.3482701043608, 1112.49613882878, 1108.07..."
1,1020,5.663806,57.401098,63.806941,"[1130.4947467286702, 1128.3418307380875, 1125...."
2,1040,4.518641,66.655180,70.094844,"[1101.9717770216187, 1074.7052510030003, 1071...."
3,1085,5.462677,52.819032,53.494464,"[1163.9734414228699, 1160.3626739233177, 1155...."
4,1104,5.631955,56.044906,57.597176,"[1128.4240845462868, 1106.169390018057, 1104.0..."


In [16]:
test.to_csv('NN_feature_window_1_overfit.csv', index = False)